# EEG Classification using CNN+LSTM model
### (And hyperparameter  Grid Search using GridSearchCV from sklearn)

In [2]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import Callback, ProgbarLogger, BaseLogger
from keras import backend as K
from keras.regularizers import l1_l2

from src import data as dt

/home/moskaleona/anaconda2/envs/tfcpu/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
path_to_data = '/home/moskaleona/alenadir/data/rawData' #'C:/Users/alena/Desktop/homed/laba/data/rawData' 

In [4]:
data = dt.DataBuildClassifier(path_to_data).get_data([25, 33], shuffle=True, random_state=1, resample_to=128, windows=[(0.2, 0.5)],baseline_window=(0.2, 0.3))

In [5]:
print('Percentage of target class: %f %%'%(data[33][1].mean()*100))
print('Percentage of target class: %f %%'%(data[25][1].mean()*100))

Percentage of target class: 64.027539 %
Percentage of target class: 71.732523 %


In [6]:
print (data[33][0].shape)
print (data[33][1].shape)

(581, 39, 19)
(581,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data[33][0], data[33][1], test_size=0.2, stratify=data[33][1], random_state=108)

In [8]:
from sklearn.metrics import roc_auc_score
import logging

class LossMetricHistory(Callback):
    def __init__(self, validation_data=(), verbose=1):
        super(LossMetricHistory, self).__init__()
        self.x_val, self.y_val = validation_data
        self.verbose = verbose
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.setLevel(logging.INFO)
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter("%(message)s")
        console.setFormatter(formatter)
        if len(self.logger.handlers) > 0:
            self.logger.handlers = []
        self.logger.addHandler(console)
            
    
    def on_train_begin(self, logs={}):
        self.logger.info("Training began")
        self.losses = []
        self.val_losses = []
        self.accs = []
        self.val_accs = []
        self.aucs = []
    
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accs.append(logs.get('acc'))
        if self.x_val is not None and self.y_val is not None: 
            self.val_losses.append(logs.get('val_loss'))
            self.val_accs.append(logs.get('val_acc'))
            self.y_pred = self.model.predict_proba(self.x_val, verbose=0)
            self.aucs.append(roc_auc_score(self.y_val, self.y_pred))
            self.logger.info("epoch %d results: train loss = %.6f, val loss = %.6f"%(epoch + 1, self.losses[-1], self.val_losses[-1]) + 
                             "\n\t\t\tacc = %.6f, val acc = %.6f"%(self.accs[-1], self.val_accs[-1]) +
                             "\n\t\t\tauc = %.6f"%(self.aucs[-1]))
        else:
            self.logger.info("epoch %d results: train loss = %.6f"%(epoch + 1, self.losses[-1]) + 
                             "\n\t\t\tacc = %.6f"%(self.accs[-1]))
        

In [14]:
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
class CnnLstmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, loss='binary_crossentropy', n_filters=10, n_lstm=30, n_iter=150, batch_size=10,
                 learning_rate=0.001, l1=0., l2=0.0, dropout=0., dropout_lstm=0., recurrent_dropout=0., threshold=0.5):
        self.loss = loss
        self.n_lstm = n_lstm
        self.n_filters = n_filters
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.l1 = l1
        self.l2 = l2
        self.dropout = dropout
        self.dropout_lstm = dropout_lstm
        self.recurrent_dropout = recurrent_dropout
        self.threshold = threshold
    
    def _make_test_model(self, input_shape):
        self.model = Sequential()
        self.model.add(Dense(1, input_shape=(741,), activation='sigmoid'))
        
    def _make_model(self, input_shape):
        batch_input_shape = (None, input_shape[1], input_shape[2])
        self.model = Sequential()
        self.model.add(Conv1D(self.n_filters, self.kernel_size_, batch_input_shape=batch_input_shape,
                         activation='relu', kernel_regularizer=l1_l2(self.l1, self.l2)))
        self.model.add(Dropout(self.dropout))
        self.model.add(LSTM(self.n_lstm,
                       dropout=self.dropout_lstm, recurrent_dropout=self.recurrent_dropout))
        self.model.add(Dense(1, activation='sigmoid'))
    
    def fit(self, X_train, y_train, X_val=None, y_val=None, verbose=2):
        # TODO: check the parameters
        self.kernel_size_ = X_train.shape[2]
        #self._make_test_model(X_train.shape)
        self._make_model(X_train.shape)
        self.optimizer_ = RMSprop(lr=self.learning_rate)
        self.model.compile(loss=self.loss, optimizer=self.optimizer_, metrics=['acc'])

        #self.log_ = LossMetricHistory(validation_data=(X_val, y_val))#BaseLogger()
        self.hist_ = self.model.fit(X_train, y_train,
                        batch_size=self.batch_size,
                        epochs=self.n_iter) 
                        #validation_data=(X_val, y_val), verbose=verbose, callbacks=[self.log_])
        return self.hist_
    
    def predict(self, X):
        '''
        try:
            getattr(self, "kernel_size_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        '''
        proba = self.model.predict(X)
        return (proba > self.threshold).astype('int32')
    
    def predict_proba(self, X):
        '''
        try:
            getattr(self, "kernel_size_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        '''
        return self.model.predict(X)
    
    
    def score(self, X, y, scoring='auc'):
        try:
            if scoring=='auc':
                return roc_auc_score(y, self.predict_proba(X))
            elif scoring=='acc':
                return accuracy_score(y, self.predict(X))
            else:
                raise ValueError(message="No such option: '%s'. Use 'auc' or 'acc'"%str(scoring))
        except ValueError as err:
            print(err)
    

## Hyperparameter tuning

In [10]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

In [11]:
'''
parameters_grid = {
    'n_iter' : [100, 200, 300],
    'l1' : [0., 0.2, 0.4, 0.6],
    'l2' : [0., 0.2, 0.4, 0.6],
    'dropout' : [0., 0.2, 0.4, 0.6],
    'dropout_lstm' : [0., 0.2, 0.4, 0.6],
    'recurrent_dropout' : [0., 0.2, 0.4, 0.6],
}
'''
parameters_grid = {
    'n_iter' : [2,3],
    'l1' : [0., 0.2]
}

In [15]:
clf = CnnLstmClassifier(n_lstm=3, n_filters=2, batch_size=10)
cv = StratifiedShuffleSplit( n_splits=2, test_size = 0.5, random_state = 108)
gs = GridSearchCV(clf, parameters_grid, scoring=None, cv = cv)

In [16]:
gs.fit(X_train, y_train)

Epoch 1/2
232/232 [==============================] - 1s 5ms/step - loss: 0.7026 - acc: 0.5560
Epoch 2/2
232/232 [==============================] - 0s 1ms/step - loss: 0.6824 - acc: 0.6164
Epoch 1/2
232/232 [==============================] - 1s 5ms/step - loss: 0.6655 - acc: 0.6121
Epoch 2/2
232/232 [==============================] - 0s 1ms/step - loss: 0.6613 - acc: 0.6164
Epoch 1/3
232/232 [==============================] - 1s 6ms/step - loss: 0.7008 - acc: 0.4957
Epoch 2/3
232/232 [==============================] - 0s 1ms/step - loss: 0.6930 - acc: 0.5690
Epoch 3/3
232/232 [==============================] - 0s 1ms/step - loss: 0.6809 - acc: 0.5948
Epoch 1/3
232/232 [==============================] - 1s 6ms/step - loss: 0.6952 - acc: 0.5603
Epoch 2/3
232/232 [==============================] - 0s 1ms/step - loss: 0.6791 - acc: 0.6164
Epoch 3/3
232/232 [==============================] - 0s 1ms/step - loss: 0.6756 - acc: 0.6379
Epoch 1/2
232/232 [==============================] - 1s 6ms/

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=2, random_state=108, test_size=0.5,
            train_size=None),
       error_score='raise',
       estimator=CnnLstmClassifier(batch_size=10, dropout=0.0, dropout_lstm=0.0, l1=0.0,
         l2=0.0, learning_rate=0.001, loss='binary_crossentropy',
         n_filters=2, n_iter=150, n_lstm=3, recurrent_dropout=0.0,
         threshold=0.5),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_iter': [2, 3], 'l1': [0.0, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
print gs.best_estimator_
print gs.best_params_
print gs.best_score_

CnnLstmClassifier(batch_size=10, dropout=0.0, dropout_lstm=0.0, l1=0.0,
         l2=0.0, learning_rate=0.001, loss='binary_crossentropy',
         n_filters=2, n_iter=3, n_lstm=3, recurrent_dropout=0.0,
         threshold=0.5)
{'n_iter': 3, 'l1': 0.0}
0.5127961510471416
